In [7]:
import os, sys
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir,'..')))

In [8]:
from scripts.data_load import load_data


In [10]:
tellco_data = load_data("../data/Data/Copy of Week2_challenge_data_source(CSV).csv")

In [ ]:
print(tellco_data.head())

In [ ]:
print(tellco_data.isnull().sum())

In [ ]:
print(tellco_data.dtypes)